In [8]:
from qmcpy import *
import numpy as np
from pandas import *
import scipy as sc
from sympy import *

In [9]:
class Estimator:
    def __init__(self, sigma=1, mean=0, n=256, h=0.0001,a=0,b=10,m=100):
        self.sampler=Sobol(1).gen_samples(n)
        self.sigma=sigma
        self.mean=mean
        self.n=n
        self.h=h
        self.m=int(m)
        self.k=np.random.normal(self.mean, self.sigma, self.m)
        self.intervals=(b-a)/self.m
        self.a=a
        self.b=b
        #self.observations=np.random.uniform(self.a,self.b,3*self.m)
        self.observations=np.random.normal(0, 1, 3*self.m)
        #simulation of observations, other distributions (Gaussian, Bernoulli..) could be applied
        
    def density_estimation(self):
        f=zeros(self.m,1)
        for i in range (self.m):
            for p in range (self.n):
                #f[i]+=(1/self.n)*(1/self.h)*self.k[i]*((self.intervals*i-self.sampler[p])/self.h)
                #we take the intervals as observations the resulting vector f is the estimation of the distribution
                f[i]+=(1/self.n)*(1/self.h)*self.k[i]*((self.observations[3*i]-self.sampler[p])/self.h)
        return f        
    def _h(self,es):
        mu0,error0=sc.integrate.quad(self.mu0,-inf,inf)
        mu2,error2=sc.integrate.quad(self.mu2,-inf,inf)
        #f=self.density_estimation()
        f2=np.diff(es,2)
        r,error=sc.integrate.quad(f2,self.a,self.b)
        h_opt=mu0/((mu2**2)*r*self.n)
        return h_opt
        
    def mu0(self,k):
        return k**2
        
    def mu2(self,k):
        return k**3
           
    def mise(self):
        #c=sc.integrate.quad(self.mu0,-inf,inf)
        #mu2,error2=sc.integrate.quad(self.mu2,-inf,inf)
        c=np.inf
        mu2=np.inf
        est=self.density_estimation()
        f2=np.diff(est,2)
        x = np.linspace(0, self.m, self.m)
        idx=np.where((x >= self.a) & (x <= self.b))
        x_lim = x[idx]
        y_lim = np.transpose(f2[idx])
        y_lim.ravel()
        r=np.trapz(y_lim, x_lim)
        #find a different method to integrate
        b=mu2*r/4
        mise=c/(self.h*self.n)+b*self.h**4
        return mise

In [12]:
est=Estimator(h=0.01,m=100)
f=est.density_estimation()
mise=est.mise()
mean=np.mean(f)
print(mise)
print(f)
print(mean)